## Data Wrangling Report

In [1]:
#import packages that will  be used in the analysis
%matplotlib inline 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import io
import json
#I will use beautifulsoup library to load my data to my file
from bs4 import BeautifulSoup

In [3]:
#Import all dataset to be used
image_data=pd.read_csv("Data//image_prediction.csv")
twitter_enhanced = pd.read_csv("Data//twitter-archive-enhanced.csv")
tweet_api_data = pd.read_csv("Data//tweet_api_data.csv")

In [4]:
#chek the columns information on image predictions dataset
image_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [5]:
#chek the columns information on twitter archive dataset
twitter_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [6]:
#chek the columns information on twitter API dataset
tweet_api_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     2354 non-null   int64  
 1   created_at                     2354 non-null   object 
 2   id                             2354 non-null   int64  
 3   id_str                         2354 non-null   int64  
 4   full_text                      2354 non-null   object 
 5   truncated                      2354 non-null   bool   
 6   display_text_range             2354 non-null   object 
 7   entities                       2354 non-null   object 
 8   extended_entities              2073 non-null   object 
 9   source                         2354 non-null   object 
 10  in_reply_to_status_id          78 non-null     float64
 11  in_reply_to_status_id_str      78 non-null     float64
 12  in_reply_to_user_id            78 non-null     f

From the above information we find out that **Image prediction dataset** does not have any null values whereas we have null values on user and twitter ids columns on **Enhancement archive dataset**
We also have couple of columns on API twitter data with null values
Most of columns we need for analysis have data and we are going to drop some columns that we dont need and have null values and this will result to us having cleaner datasets

<a id='archive'></a>
## Twitter Archive Data Wrangling

Since this dataset have date column, we are going to format the date column to a format that we can work with in our analysis

In [9]:
#Le us start by looking at date formats on all datasets which need to be formatted to the correct format
twitter_enhanced.timestamp[:2]

0    2017-08-01 16:23:56 +0000
1    2017-08-01 00:17:27 +0000
Name: timestamp, dtype: object

In [10]:
#Convert the date column
twitter_enhanced["timestamp"]=pd.to_datetime(twitter_enhanced["timestamp"])

twitter_enhanced["timestamp"] = twitter_enhanced["timestamp"].dt.date
#After converting now check the column
twitter_enhanced.timestamp[:2]

0    2017-08-01
1    2017-08-01
Name: timestamp, dtype: object

In [11]:
#Let us confirm the column data type
twitter_enhanced.timestamp.info()

<class 'pandas.core.series.Series'>
RangeIndex: 2356 entries, 0 to 2355
Series name: timestamp
Non-Null Count  Dtype 
--------------  ----- 
2356 non-null   object
dtypes: object(1)
memory usage: 18.5+ KB


We have seen that the date has changed from timestamp to object
We need to change the data type back to date format

In [13]:
#Convert the data type back to datetime
twitter_enhanced["timestamp"]=pd.to_datetime(twitter_enhanced["timestamp"])
#Confirm the date type again
twitter_enhanced.timestamp.info()

<class 'pandas.core.series.Series'>
RangeIndex: 2356 entries, 0 to 2355
Series name: timestamp
Non-Null Count  Dtype         
--------------  -----         
2356 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 18.5 KB


The date format has now changed to date format that is the correct format of the date column for analysis

<a id='archive_clean'></a>
### Twitter Archive Data Wrangling

In [14]:
#Make the copy of the data
twitter_enhanced_v1=twitter_enhanced.copy()
twitter_enhanced_v1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   tweet_id                    2356 non-null   int64         
 1   in_reply_to_status_id       78 non-null     float64       
 2   in_reply_to_user_id         78 non-null     float64       
 3   timestamp                   2356 non-null   datetime64[ns]
 4   source                      2356 non-null   object        
 5   text                        2356 non-null   object        
 6   retweeted_status_id         181 non-null    float64       
 7   retweeted_status_user_id    181 non-null    float64       
 8   retweeted_status_timestamp  181 non-null    object        
 9   expanded_urls               2297 non-null   object        
 10  rating_numerator            2356 non-null   int64         
 11  rating_denominator          2356 non-null   int64       

In our analysis we do not need retweet and inreply columns `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` and `retweeted_status_timestamp`

In [18]:
#Columns to delete
delete_columns = ['in_reply_to_user_id', 'in_reply_to_status_id',  
           'retweeted_status_id', 'retweeted_status_user_id', 
           'retweeted_status_timestamp']
#Drop the columns
twitter_enhanced_v1 = twitter_enhanced_v1.drop(delete_columns, axis=1)
twitter_enhanced_v1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2356 non-null   int64         
 1   timestamp           2356 non-null   datetime64[ns]
 2   source              2356 non-null   object        
 3   text                2356 non-null   object        
 4   expanded_urls       2297 non-null   object        
 5   rating_numerator    2356 non-null   int64         
 6   rating_denominator  2356 non-null   int64         
 7   name                2356 non-null   object        
 8   doggo               2356 non-null   object        
 9   floofer             2356 non-null   object        
 10  pupper              2356 non-null   object        
 11  puppo               2356 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 221.0+ KB


In [19]:
#Le us start by looking at date formats on the dataset which need to be formatted to the correct format
twitter_enhanced_v1.timestamp[:2]

0   2017-08-01
1   2017-08-01
Name: timestamp, dtype: datetime64[ns]

The date format in this dataset seem to be in the correct format so there is no need of reformating the column
However when you look at the data we need to melt dog types column so that we can have dog types in one column and this will result to a tidy data that can easily be used in data analysis

In [21]:
#We need to unnest the data so that we can include dog types column as arrays in one column
twitter_enhanced_v2= twitter_enhanced_v1.melt(id_vars=["tweet_id", "timestamp","source","text","expanded_urls",
                                                       "rating_numerator","rating_denominator","name"], 
        var_name="Dog_type", 
        value_name="type_available")
twitter_enhanced_v2.sample(5)


,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,Dog_type,type_available
8470,699423671849451520,2016-02-16,"<a href=""http://twitter.com/download/iphone"" r...","""Don't ever talk to me or my son again."" ...bo...",https://twitter.com/dog_rates/status/699423671...,10,10,None,puppo,None
790,773922284943896577,2016-09-08,"<a href=""http://twitter.com/download/iphone"" r...","This is Heinrich (pronounced ""Pat""). He's a Bo...",https://twitter.com/dog_rates/status/773922284...,12,10,Heinrich,doggo,None
8338,709449600415961088,2016-03-14,"<a href=""http://twitter.com/download/iphone"" r...",Meet Karma. She's just a head. Lost body durin...,https://twitter.com/dog_rates/status/709449600...,10,10,Karma,puppo,None
585,800018252395122689,2016-11-19,"<a href=""http://twitter.com/download/iphone"" r...",Here's a doggo doin a struggle. 11/10 much det...,https://twitter.com/dog_rates/status/800018252...,11,10,None,doggo,doggo
3361,747651430853525504,2016-06-28,"<a href=""http://twitter.com/download/iphone"" r...",Other pupper asked not to have his identity sh...,NaN,12,10,None,floofer,None


In [23]:
#Check the numerator ratings
twitter_enhanced_v2.rating_numerator.value_counts()

12      2232
11      1856
10      1844
13      1404
9        632
8        408
7        220
14       216
5        148
6        128
3         76
4         68
2         36
1         36
75         8
15         8
420        8
0          8
80         4
144        4
17         4
26         4
20         4
121        4
143        4
44         4
60         4
45         4
50         4
99         4
204        4
1776       4
165        4
666        4
27         4
182        4
24         4
960        4
84         4
88         4
Name: rating_numerator, dtype: int64

From above we find out that there are some ratings that are more than 100 ratings for each column and investigate data


In [24]:
twitter_enhanced_v3=twitter_enhanced_v2.query("rating_denominator > 100")
twitter_enhanced_v3.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,Dog_type,type_available
902,758467244762497024,2016-07-28,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,https://twitter.com/dog_rates/status/758467244...,165,150,None,doggo,None
1120,731156023742988288,2016-05-13,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to this unbelievably well behaved sq...,https://twitter.com/dog_rates/status/731156023...,204,170,this,doggo,None
1634,684225744407494656,2016-01-05,"<a href=""http://twitter.com/download/iphone"" r...","Two sneaky puppers were not initially seen, mo...",https://twitter.com/dog_rates/status/684225744...,143,130,None,doggo,None
1635,684222868335505415,2016-01-05,"<a href=""http://twitter.com/download/iphone"" r...",Someone help the girl is being mugged. Several...,https://twitter.com/dog_rates/status/684222868...,121,110,None,doggo,None
1779,677716515794329600,2015-12-18,"<a href=""http://twitter.com/download/iphone"" r...",IT'S PUPPERGEDDON. Total of 144/120 ...I think...,https://twitter.com/dog_rates/status/677716515...,144,120,None,doggo,None


From this we can see that we almost all of the tweets that have ratings more than 100 have no name of the dog. Also there is one tweet with ratings 204 that has dog name as `this` which I also consider as no name. `this` is not a correct name of the dog conclusing that all tweets with more than 100 ratings have no dog names

<a id='image'></a>
### Image Predictions Data Wrangling

In [27]:
image_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


We can see that there is no null value in any column

In [28]:
image_data.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01
